# NOTEBOOK FOR TESTING

### IMPORT LIBRARIES

In [44]:
import sqlite3  # library to import      
import time 
import re   
import nltk # library to import 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize 
from nltk.corpus.reader.wordnet import NOUN, VERB, ADJ, ADV

from collections import Counter 
from simhash import Simhash

nltk.download('stopwords')
nltk.download('wordnet') 
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')  
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utente\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\utente\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\utente\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\utente\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\utente\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\utente\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already u

True

### PREPROCESS TEXT

In [37]:
def preprocess_sbagliato(text):
    #   Set of stopwords 
    stop_words = set(stopwords.words('english')) 
    #   Lemmatizer 
    lemmatizer = WordNetLemmatizer()
    #   potrebbe essere utile cambiare il set di stopwords a seconda della lingua della pagina
    #   devo capire come fare
    #   Tokenize the content       
    words = re.findall(r'\w+', text.lower())

    #   Remove the stop words from the set of words 
    words = [w for w in words if w.isalpha() and w not in stop_words]

    #   Apply lemmatization (keep lemmatization or apply stemming?)
    words = [lemmatizer.lemmatize(word) for word in words]

    #   Return the set of words preprocessed 
    return words 

Example

In [ ]:
text = "The man suspected of opening fire on two Minnesota legislators and their spouses on 14 June, killing one legislator and her husband, was apprehended late on Sunday night and charged with two counts of murder and two of attempted murder, the state’s governor, Tim Walz, said at a news conference. Vance Boelter, 57, is suspected of fatally shooting the Democratic state representative Melissa Hortman and her husband, Mark, at their residence early on Saturday. Boelter is also suspected of shooting the state senator John Hoffman and his wife, Yvette, at their home, seriously injuring them. “One man’s unthinkable actions have altered the state of Minnesota,” the state’s governor, Tim Walz, said at a news conference. Boelter was arrested in a rural area in Sibley County, southwest of Minneapolis, according to police, who added that he was armed when he was taken into custody. A criminal complaint unsealed Sunday night said Boelter faces two counts of second-degree murder and two counts of attempted second-degree murder in the deaths of the Hortmans and the wounding of Hoffman and his wife."

words = preprocess_sbagliato(text) 

print(words)


['man', 'suspected', 'opening', 'fire', 'two', 'minnesota', 'legislator', 'spouse', '14', 'june', 'killing', 'one', 'legislator', 'husband', 'apprehended', 'late', 'sunday', 'night', 'charged', 'two', 'count', 'murder', 'two', 'attempted', 'murder', 'state', 'governor', 'tim', 'walz', 'said', 'news', 'conference', 'vance', 'boelter', '57', 'suspected', 'fatally', 'shooting', 'democratic', 'state', 'representative', 'melissa', 'hortman', 'husband', 'mark', 'residence', 'early', 'saturday', 'boelter', 'also', 'suspected', 'shooting', 'state', 'senator', 'john', 'hoffman', 'wife', 'yvette', 'home', 'seriously', 'injuring', 'one', 'man', 'unthinkable', 'action', 'altered', 'state', 'minnesota', 'state', 'governor', 'tim', 'walz', 'said', 'news', 'conference', 'boelter', 'arrested', 'rural', 'area', 'sibley', 'county', 'southwest', 'minneapolis', 'according', 'police', 'added', 'armed', 'taken', 'custody', 'criminal', 'complaint', 'unsealed', 'sunday', 'night', 'said', 'boelter', 'face', 't

Come si può vedere, non viene fatto il **lemmatization** bene, poiché non riconosce se una parola è un sostantivo oppure un verbo.

Per sistemare questa cosa, introduco il **POS tagging**, che è il processo di assegnare a ciascuna parola di un testo il suo ruolo grammaticale. 

Sistemo quindi il codice. 

In [45]:
def get_wordnet_pos(treebank_tag): 
    if treebank_tag.startswith('J'):
        return ADJ
    elif treebank_tag.startswith('V'):
        return VERB
    elif treebank_tag.startswith('N'):
        return NOUN
    elif treebank_tag.startswith('R'):
        return ADV
    else:
        return NOUN  # default

def preprocess(text): 
    words = word_tokenize(text.lower())
    #   Set of stopwords 
    stop_words = set(stopwords.words('english')) 
    #   Lemmatizer 
    lemmatizer = WordNetLemmatizer()

    number_words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']

    #   Remove stop words and punctuation 
    words = [w for w in words if w.isalpha() and w not in stop_words and w not in number_words]

    tagged_words = pos_tag(words)

    #   Lemmatization is done using the correct grammatical type 
    lemmatized_words = [lemmatizer.lemmatize(w, get_wordnet_pos(t)) for w, t in tagged_words]

    return lemmatized_words


Riprovo l'esempio di prima

In [46]:
text = "The man suspected of opening fire on two Minnesota legislators and their spouses on 14 June, killing one legislator and her husband, was apprehended late on Sunday night and charged with two counts of murder and two of attempted murder, the state’s governor, Tim Walz, said at a news conference. Vance Boelter, 57, is suspected of fatally shooting the Democratic state representative Melissa Hortman and her husband, Mark, at their residence early on Saturday. Boelter is also suspected of shooting the state senator John Hoffman and his wife, Yvette, at their home, seriously injuring them. “One man’s unthinkable actions have altered the state of Minnesota,” the state’s governor, Tim Walz, said at a news conference. Boelter was arrested in a rural area in Sibley County, southwest of Minneapolis, according to police, who added that he was armed when he was taken into custody. A criminal complaint unsealed Sunday night said Boelter faces two counts of second-degree murder and two counts of attempted second-degree murder in the deaths of the Hortmans and the wounding of Hoffman and his wife."

words = preprocess(text) 

print(words)

['man', 'suspect', 'opening', 'fire', 'minnesota', 'legislator', 'spouses', 'june', 'kill', 'legislator', 'husband', 'apprehend', 'late', 'sunday', 'night', 'charge', 'count', 'murder', 'attempt', 'murder', 'state', 'governor', 'tim', 'walz', 'say', 'news', 'conference', 'vance', 'boelter', 'suspect', 'fatally', 'shoot', 'democratic', 'state', 'representative', 'melissa', 'hortman', 'husband', 'mark', 'residence', 'early', 'saturday', 'boelter', 'also', 'suspect', 'shoot', 'state', 'senator', 'john', 'hoffman', 'wife', 'yvette', 'home', 'seriously', 'injure', 'man', 'unthinkable', 'action', 'alter', 'state', 'minnesota', 'state', 'governor', 'tim', 'walz', 'say', 'news', 'conference', 'boelter', 'arrest', 'rural', 'area', 'sibley', 'county', 'southwest', 'minneapolis', 'accord', 'police', 'add', 'arm', 'take', 'custody', 'criminal', 'complaint', 'unseal', 'sunday', 'night', 'say', 'boelter', 'face', 'count', 'murder', 'count', 'attempt', 'murder', 'death', 'hortmans', 'wound', 'hoffman

In [47]:
tf = Counter(words)

print(tf)

Counter({'state': 5, 'murder': 4, 'boelter': 4, 'suspect': 3, 'count': 3, 'say': 3, 'man': 2, 'minnesota': 2, 'legislator': 2, 'husband': 2, 'sunday': 2, 'night': 2, 'attempt': 2, 'governor': 2, 'tim': 2, 'walz': 2, 'news': 2, 'conference': 2, 'shoot': 2, 'hoffman': 2, 'wife': 2, 'opening': 1, 'fire': 1, 'spouses': 1, 'june': 1, 'kill': 1, 'apprehend': 1, 'late': 1, 'charge': 1, 'vance': 1, 'fatally': 1, 'democratic': 1, 'representative': 1, 'melissa': 1, 'hortman': 1, 'mark': 1, 'residence': 1, 'early': 1, 'saturday': 1, 'also': 1, 'senator': 1, 'john': 1, 'yvette': 1, 'home': 1, 'seriously': 1, 'injure': 1, 'unthinkable': 1, 'action': 1, 'alter': 1, 'arrest': 1, 'rural': 1, 'area': 1, 'sibley': 1, 'county': 1, 'southwest': 1, 'minneapolis': 1, 'accord': 1, 'police': 1, 'add': 1, 'arm': 1, 'take': 1, 'custody': 1, 'criminal': 1, 'complaint': 1, 'unseal': 1, 'face': 1, 'death': 1, 'hortmans': 1, 'wound': 1})


### SIMHASH 

In [51]:
def compute_fingerprint(text):
    words = preprocess(text)
    return Simhash(words).value

In [52]:
print(compute_fingerprint(text))

2420779509525056338


Adesso provo a verificare se due contenuti sono "near duplicates"